In [ ]:
!pip install comet_ml > /dev/null 2>&1
import comet_ml
# TODO: ENTER YOUR API KEY HERE!! instructions above
COMET_API_KEY = "5i0jFaHpBypwOFgN3tQlxtMoJ"

# Import PyTorch and other relevant libraries
import torch
import torch.nn as nn
import torch.optim as optim

# Download and import the MIT Introduction to Deep Learning package
!pip install mitdeeplearning --quiet
import mitdeeplearning as mdl

# Import all remaining packages
import numpy as np
import os
import time
import functools
from IPython import display as ipythondisplay
from tqdm import tqdm
from scipy.io.wavfile import write
!apt-get install abcmidi timidity > /dev/null 2>&1


# Check that we are using a GPU, if not switch runtimes
#   using Runtime > Change Runtime Type > GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
assert COMET_API_KEY != "", "Please insert your Comet API Key"

In [ ]:
# Download the dataset
songs = mdl.lab1.load_training_data()

# Print one of the songs to inspect it in greater detail!
example_song = songs[0]
print("\nExample song: ")
print(example_song)

In [ ]:
# Download the dataset
songs = mdl.lab1.load_training_data()

# let's try on the songs
Example_song = songs[0]

print("Example song\n", Example_song)

We can easily convert a song in ABC notation to an audio waveform and play it back. Be patient for this conversion to run, it can take some time.

In [ ]:
# Convert the ABC notation to audio file and ilsten to it
mdl.lab1.play_song(Example_song)

One important thing to think about is that this notation of music does not simply contain information on the notes being played, but additionally there is meta information such as the song title, key, and tempo. How does the number of different characters that are present in the text file impact the complexity of the learning problem? This will become important soon, when we generate a numerical representation for the text data.

In [ ]:
# Join our list of song strings into a single string containing all songs
songs_joined = '\n\n'.join(songs)

# Find all unique characters in the joined string
vocab = sorted(set(songs_joined))
print(f"There is {len(vocab)} unique characters in the dataset")

#  Process the dataset for the learning task

Let's take a step back and consider our prediction task. We're trying to train an RNN model to learn patterns in ABC music, and then use this model to generate (i.e., predict) a new piece of music based on this learned information.

Breaking this down, what we're really asking the model is: given a character, or a sequence of characters, what is the most probable next character? We'll train the model to perform this task.

To achieve this, we will input a sequence of characters to the model, and train the model to predict the output, that is, the following character at each time step. RNNs maintain an internal state that depends on previously seen elements, so information about all characters seen up until a given moment will be taken into account in generating the prediction.


## Vectorize the text

Before we begin training our RNN model, we'll need to create a numerical representation of our text-based dataset. To do this, we'll generate two lookup tables: one that maps characters to numbers, and a second that maps numbers back to characters. Recall that we just identified the unique characters present in the text.

In [ ]:
### Define numerical representation of text ###

# Create a mapping from character to unique index.
# For example, to get the index of the character "d",
#  we can evaluate `char2idx["d"]`.

char2idx = {u : i for i, u in enumerate(vocab)}

# Create a mapping from indices to characters. This is
#  the inverse of char2idx and allows us to convert back
#  from unique index to the character in our vocabulary.

idx2char = np.array(vocab)


This gives us an integer representation for each character. Observe that the unique characters (i.e., our vocabulary) in the text are mapped as indices from 0 to len(unique). Let's take a peek at this numerical representation of our dataset:

In [ ]:
print('{')
for i, char in enumerate(char2idx):
    if i == 20:
        break
    print(f'  {repr(char):4s}: {char2idx[char]:3d},')
print('  ...\n}')


In [ ]:
### Vectorize the songs string ###

def vectorize_string(string):
  return np.array([char2idx[c] for c in string], dtype=np.int32)

vectorized_songs = vectorize_string(songs_joined)

We can also look at how the first part of the text is mapped to an integer representation:

In [ ]:
print ('{} ---- characters mapped to int ----> {}'.format(repr(songs_joined[:10]), vectorized_songs[:10]))

# check that vectorized_songs is a numpy array
assert isinstance(vectorized_songs, np.ndarray), "returned result should be a numpy array"

#Create training examples and targets
Our next step is to actually divide the text into example sequences that we'll use during training. Each input sequence that we feed into our RNN will contain seq_length characters from the text. We'll also need to define a target sequence for each input sequence, which will be used in training the RNN to predict the next character. For each input, the corresponding target will contain the same length of text, except shifted one character to the right.

To do this, we'll break the text into chunks of seq_length+1. Suppose seq_length is 4 and our text is "Hello". Then, our input sequence is "Hell" and the target sequence is "ello".

The batch method will then let us convert this stream of character indices to sequences of the desired size.

In [ ]:
### Batch definition to create training examples ###

def get_batch(vectorized_songs, seq_length, batch_size):

      # the length of the vectorized songs string
      n = vectorized_songs.shape[0] - 1

      # randomly choose the starting indices for the examples in the training batch
      idx = np.random.choice(n - seq_length, batch_size, replace = False)

      # construct a list of input sequences for the training batch
      input_batch = [vectorized_songs[i : i + seq_length] for i in idx]

      # construct a list of input sequences for the training batch
      output_batch = [vectorized_songs[i + 1 : i + seq_length + 1] for i in idx]

      # Convert the input and output batches to tensors
      x_batch = torch.tensor(input_batch, dtype=torch.long)
      y_batch = torch.tensor(output_batch, dtype=torch.long)

      return x_batch, y_batch

# Perform some simple tests to make sure your batch function is working properly!
test_args = (vectorized_songs, 10, 2)
x_batch, y_batch = get_batch(*test_args)
assert x_batch.shape == (2, 10), "x_batch shape is incorrect"
assert y_batch.shape == (2, 10), "y_batch shape is incorrect"
print("Batch function works correctly!")

For each of these vectors, each index is processed at a single time step. So, for the input at time step 0, the model receives the index for the first character in the sequence, and tries to predict the index of the next character. At the next timestep, it does the same thing, but the RNN considers the information from the previous step, i.e., its updated state, in addition to the current input.

We can make this concrete by taking a look at how this works over the first several characters in our text:

In [ ]:
x_batch, y_batch = get_batch(vectorized_songs, seq_length=5, batch_size=1)

print("Time step | Input | Expected Output")
print("-" * 30)

for i, (inp, tgt) in enumerate(zip(x_batch[0], y_batch[0])):
    inp_char = idx2char[inp.item()]
    tgt_char = idx2char[tgt.item()]
    print(f"{i:9} | {inp:5} ({inp_char}) | {tgt:15} ({tgt_char})")


#  The Recurrent Neural Network (RNN) model
Now we're ready to define and train an RNN model on our ABC music dataset, and then use that trained model to generate a new song. We'll train our RNN using batches of song snippets from our dataset, which we generated in the previous section.

The model is based off the LSTM architecture, where we use a state vector to maintain information about the temporal relationships between consecutive characters. The final output of the LSTM is then fed into a fully connected linear nn.Linear layer where we'll output a softmax over each character in the vocabulary, and then sample from this distribution to predict the next character.

As we introduced in the first portion of this lab, we'll be using PyTorch's nn.Module to define the model. Three components are used to define the model:

nn.Embedding: This is the input layer, consisting of a trainable lookup table that maps the numbers of each character to a vector with embedding_dim dimensions.
nn.LSTM: Our LSTM network, with size hidden_size.
nn.Linear: The output layer, with vocab_size outputs.
Drawing
https://camo.githubusercontent.com/7d0de1b1f69f93e99d0d0deb39f201c51326016edc90f26066ec3e9c56186e1b/68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f4d4954446565704c6561726e696e672f696e74726f746f646565706c6561726e696e672f6d61737465722f6c6162312f696d672f6c73746d5f756e726f6c6c65642d30312d30312e706e67

# Define the RNN model
Let's define our model as an nn.Module.

In [ ]:
import torch
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size

        # Layer 1: Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # Layer 2: LSTM layer
        self.lstm = nn.LSTM(input_size=embedding_dim,
                            hidden_size=hidden_size,
                            num_layers=1,
                            batch_first=True)

        # Layer 3: Linear layer to project hidden state to vocabulary size
        self.fc = nn.Linear(hidden_size, vocab_size)

    def init_hidden(self, batch_size, device):
        # Initialize hidden state and cell state with zeros
        return (torch.zeros(1, batch_size, self.hidden_size).to(device),
                torch.zeros(1, batch_size, self.hidden_size).to(device))

    def forward(self, x, state=None, return_state=False):
        # Convert input indices to embeddings
        x = self.embedding(x)  # [batch_size, seq_len] -> [batch_size, seq_len, embedding_dim]

        # Initialize hidden state if not provided
        if state is None:
            state = self.init_hidden(x.size(0), x.device)

        # Forward pass through LSTM
        out, state = self.lstm(x, state)  # out: [batch_size, seq_len, hidden_size]

        # Map LSTM outputs to vocabulary space
        out = self.fc(out)  # [batch_size, seq_len, vocab_size]

        return out if not return_state else (out, state)


The time has come! Let's instantiate the model!

In [ ]:
# Instantiate the model! Build a simple model with default hyperparameters. You
# will get the chance to change these later.
vocab_size = len(vocab)
embedding_dim = 256
hidden_size = 1024
batch_size = 8

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = LSTMModel(vocab_size, embedding_dim, hidden_size).to(device)

# print out the summary of the model
print(model)

# Test out the RNN model
It's always a good idea to run a few simple checks on our model to see that it behaves as expected.

We can quickly check the layers in the model, the shape of the output of each of the layers, the batch size, and the dimensionality of the output. Note that the model can be run on inputs of any length.

In [ ]:
# test the model with sample data
x, y = get_batch(vectorized_songs, seq_length=100, batch_size=32)
x = x.to(device)
y = y.to(device)

pred = model(x)

print("Input shape:      ", x.shape, '# (batch_size, seq_length)')
print("Prediction shape: ", pred.shape, '# (batch_size, seq_length, vocab_size)')

# Predictions from the untrained model
Let's take a look at what our untrained model is predicting.

To get actual predictions from the model, we sample from the output distribution, which is defined by a torch.softmax over our character vocabulary. This will give us actual character indices. This means we are using a categorical distribution to sample over the example prediction. This gives a prediction of the next character (specifically its index) at each timestep. torch.multinomial samples over a categorical distribution to generate predictions.

Note here that we sample from this probability distribution, as opposed to simply taking the argmax, which can cause the model to get stuck in a repetitive loop.

Let's try this sampling out for the first example in the batch.

In [ ]:
sampled_indices = torch.multinomial(torch.softmax(pred[0], dim = -1), num_samples=1) # dim = -1 for vocab dim and num_samples = 1 step one
sampled_indices = sampled_indices.squeeze().cpu().numpy()
sampled_indices

We can now decode these to see the text predicted by the untrained model:

In [ ]:
print("Input: \n", repr("".join(idx2char[x[0].cpu()])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices])))

As you can see, the text predicted by the untrained model is pretty nonsensical! How can we do better? Well, we can train the network!

# Training the model: loss and training operations
Now it's time to train the model!

At this point, we can think of our next character prediction problem as a standard classification problem. Given the previous state of the RNN, as well as the input at a given time step, we want to predict the class of the next character -- that is, to actually predict the next character.

To train our model on this classification task, we can use a form of the crossentropy loss (i.e., negative log likelihood loss). Specifically, we will use PyTorch's CrossEntropyLoss, as it combines the application of a log-softmax (LogSoftmax) and negative log-likelihood (NLLLoss in a single class and accepts integer targets for categorical classification tasks. We will want to compute the loss using the true targets -- the labels -- and the predicted targets -- the logits.

Let's define a function to compute the loss, and then use that function to compute the loss using our example predictions from the untrained model.

In [ ]:
cross_entropy = nn.CrossEntropyLoss()

def compute_loss(labels, logits):
    """
    Inputs:
      labels: (batch_size, sequence_length)
      logits: (batch_size, sequence_length, vocab_size)

    Output:
      loss: scalar cross entropy loss over the batch and sequence length
    """
    # Flatten labels
    batched_labels = labels.view(-1)

    # Flatten logits
    batched_logits = logits.view(-1, logits.size(-1))

    # compute cross_entropy loss
    loss = cross_entropy(batched_logits, batched_labels)
    return loss


In [ ]:
### compute the loss on the predictions from the untrained model from earlier. ###
y.shape # (batch_size, seq_len)
pred.shape # (batch_size, seq_len, vocab_size)

example_batch_loss = compute_loss(y, pred)

print(f"Prediction shape: {pred.shape} # (batch_size, sequence_length, vocab_size)")
print(f"scalar_loss:      {example_batch_loss.mean().item()}")

Let's start by defining some hyperparameters for training the model. To start, we have provided some reasonable values for some of the parameters. It is up to you to use what we've learned in class to help optimize the parameter selection here!

In [ ]:
vocab_size = len(vocab)

params = dict (
    num_training_iteration = 3000, # increase this to train longer
    batch_size = 8, # increase this the to train faster
    seq_len = 100, # the len of each input sequence
    learning_rate = 5e-2,
    embedding_dim = 256,
    hidden_size = 1024,
)
# Checkpoint location:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "my_ckpt")
os.makedirs(checkpoint_dir, exist_ok=True)


Having defined our hyperparameters we can set up for experiment tracking with Comet. Experiment are the core objects in Comet and will allow us to track training and model development. Here we have written a short function to create a new Comet experiment. Note that in this setup, when hyperparameters change, you can run the create_experiment() function to initiate a new experiment. All experiments defined with the same project_name will live under that project in your Comet interface.

In [ ]:
### Create a Comet experiment to track our training run ###

def create_experiment():
  # end any prior experiments
  if 'experiment' in locals():
    experiment.end()

  # initiate the comet experiment for tracking
  experiment = comet_ml.Experiment(
                  api_key=COMET_API_KEY,
                  project_name="6S191_Lab1_Part2")
  # log our hyperparameters, defined above, to the experiment
  for param, value in params.items():
    experiment.log_parameter(param, value)
  experiment.flush()

  return experiment

Now, we are ready to define our training operation -- the optimizer and duration of training -- and use this function to train the model. You will experiment with the choice of optimizer and the duration for which you train your models, and see how these changes affect the network's output. Some optimizers you may like to try are Adam and Adagrad.

First, we will instantiate a new model and an optimizer, and ready them for training. Then, we will use loss.backward(), enabled by PyTorch's autograd method, to perform the backpropagation. Finally, to update the model's parameters based on the computed gradients, we will utake a step with the optimizer, using optimizer.step().

We will also generate a print-out of the model's progress through training, which will help us easily visualize whether or not we are minimizing the loss.

In [ ]:
# Instantiate the model
model = LSTMModel(vocab_size, params['embedding_dim'], params['hidden_size'])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr = params['learning_rate'])

# loss function
cross_entropy = nn.CrossEntropyLoss()

# traning step
def training_step(x, y):
  model.train()
  optimizer.zero_grad()
  y_pred = model(x)

  loss = compute_loss(y, y_pred)
  loss.backward()
  optimizer.step()

  return loss

# Training loop
history = []

for iter in tqdm(range(params["num_training_iteration"])):
    x_batch, y_batch = get_batch(vectorized_songs, params["seq_len"], params["batch_size"])
    x_batch = torch.tensor(x_batch, dtype=torch.long).to(device)
    y_batch = torch.tensor(y_batch, dtype=torch.long).to(device)

    loss = training_step(x_batch, y_batch)

    history.append(loss.item())

    if iter % 100 == 0:
        print(f"Step {iter}, Loss: {loss.item():.4f}")
        torch.save(model.state_dict(), checkpoint_prefix)

# Save final model
torch.save(model.state_dict(), checkpoint_prefix)
print("Training finished and model saved.")

#  Generate music using the RNN model
Now, we can use our trained RNN model to generate some music! When generating music, we'll have to feed the model some sort of seed to get it started (because it can't predict anything without something to start with!).

Once we have a generated seed, we can then iteratively predict each successive character (remember, we are using the ABC representation for our music) using our trained RNN. More specifically, recall that our RNN outputs a softmax over possible successive characters. For inference, we iteratively sample from these distributions, and then use our samples to encode a generated song in the ABC format.

Then, all we have to do is write it to a file and listen!

The prediction procedure
Now, we're ready to write the code to generate text in the ABC music format:

Initialize a "seed" start string and the RNN state, and set the number of characters we want to generate.

Use the start string and the RNN state to obtain the probability distribution over the next predicted character.

Sample from multinomial distribution to calculate the index of the predicted character. This predicted character is then used as the next input to the model.

At each time step, the updated RNN state is fed back into the model, so that it now has more context in making the next prediction. After predicting the next character, the updated RNN states are again fed back into the model, which is how it learns sequence dependencies in the data, as it gets more information from the previous predictions.

https://camo.githubusercontent.com/85f6271761cc48c50de28a63261c5922fb4c7f03cca29e899b2e2e3bbebf615a/68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f4d4954446565704c6561726e696e672f696e74726f746f646565706c6561726e696e672f6d61737465722f6c6162312f696d672f6c73746d5f696e666572656e63652e706e67

Complete and experiment with this code block (as well as some of the aspects of network definition and training!), and see how the model performs. How do songs generated after training with a small number of epochs compare to those generated after a longer duration of training?

In [ ]:
def generate_text(model, start_string, generation_length=1000):
    model.eval()  # Set model to evaluation mode
    device = next(model.parameters()).device  # Get device (CPU/GPU)

    # Convert the start string to indices
    input_idx = [char2idx[c] for c in start_string]
    input_idx = torch.tensor([input_idx], dtype=torch.long).to(device)  # shape: [1, len(start_string)]

    # Initialize the LSTM hidden state and cell state
    state = model.init_hidden(batch_size=1, device=device)

    # Store generated characters
    text_generated = []

    tqdm._instances.clear()  # clear old tqdm instances

    for _ in tqdm(range(generation_length)):
        # Forward pass
        predictions, state = model(input_idx, state, return_state=True)

        # Take only the last character's predictions
        predictions = predictions[:, -1, :]  # shape: [1, vocab_size]

        # Convert logits to probabilities
        probs = torch.softmax(predictions, dim=-1)

        # Sample the next character from the distribution
        next_idx = torch.multinomial(probs, num_samples=1)  # shape: [1,1]

        # Append predicted character
        text_generated.append(idx2char[next_idx.item()])

        # Prepare next input (keep batch dim = 1, seq_len = 1)
        input_idx = next_idx  # shape: [1,1]

    return start_string + "".join(text_generated)


# Example usage
generated_song = generate_text(model, start_string="X:", generation_length=500)
print("Generated music:\n")
print(generated_song)


In [ ]:
# Generate ABC format text using the trained model
# Start string "X:" because ABC files usually start with X: field
generated_text = generate_text(model, start_string="X:", generation_length=1000)

# Print the generated text
print("Generated ABC music text:\n")
print(generated_text)


# Play back the generated music!
We can now call a function to convert the ABC format text to an audio file, and then play that back to check out our generated music! Try training longer if the resulting song is not long enough, or re-generating the song!

We will save the song to Comet -- you will be able to find your songs under the Audio and Assets & Artifacts pages in your Comet interface for the project. Note the log_asset() documentation, where you will see how to specify file names and other parameters for saving your assets.

In [ ]:
### Play back generated songs ###

generated_songs = mdl.lab1.extract_song_snippet(generated_text)

for i, song in enumerate(generated_songs):
  # Synthesize the waveform from a song
  waveform = mdl.lab1.play_song(song)

  # If its a valid song (correct syntax), lets play it!
  if waveform:
    print("Generated song", i)
    ipythondisplay.display(waveform)

    numeric_data = np.frombuffer(waveform.data, dtype=np.int16)
    wav_file_path = f"output_{i}.wav"
    write(wav_file_path, 88200, numeric_data)

    # save your song to the Comet interface -- you can access it there
    experiment = create_experiment()
    experiment.log_asset(wav_file_path)

In [ ]:
# when done, end the comet experiment
experiment.end()